# Video Pipeline

1. Download video(s)
2. Metadata collection
3. Frame/Clip decomposition
4. NudeNet

## Dependencies

In [12]:
!pip3 install youtube_dl
!pip3 install beautifulsoup4
!pip3 install requests
!pip3 install numpy
!pip3 install nudenet
!pip3 install tensorflow
!pip3 install opencv-python

In [14]:
from __future__ import unicode_literals
from bs4 import BeautifulSoup as bs
from nudenet import NudeDetector

import youtube_dl
import subprocess
import math
import shlex
import requests
import argparse
import os
import sys
import subprocess
import cv2  
import json
import tensorflow as tf
import warnings
import glob
import uuid

Using TensorFlow backend.


-------------------

## 1. Download video

In [7]:
ydl_opts = {
    'format': 'best',       
    'outtmpl': '%(id)s'+'.mp4'
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    meta = ydl.extract_info('https://www.youtube.com/watch?v=XVNQSdAWVGo', download=True)
    
    print('Done! Video downloaded.')


[youtube] XVNQSdAWVGo: Downloading webpage
[youtube] XVNQSdAWVGo: Downloading video info webpage
[download] XVNQSdAWVGo.mp4 has already been downloaded
[download] 100% of 12.62MiB
Done! Video downloaded.


---------

## 2. Metadata collection

In [8]:
youtube_id = meta['id']
uploader = meta['uploader']
uploader_id = meta['uploader_id']
uploader_url = meta['uploader_url']
channel_id = meta['channel_id']
channel_url = meta['channel_url']
upload_date = meta['upload_date']
license = meta['license']
creator = meta['creator']
title = meta['title']
alt_title = meta['alt_title']
thumbnail = meta['thumbnail']
description = meta['description']
categories = meta['categories']
tags = meta['tags']
subtitles = meta['subtitles']
automatic_captions = meta['automatic_captions']
duration = meta['duration']
age_limit = meta['age_limit']
annotations = meta['annotations']
chapters = meta['chapters']
webpage_url = meta['webpage_url']
view_count = meta['view_count']
like_count = meta['like_count']
dislike_count = meta['dislike_count']
average_rating = meta['average_rating']

print('')
print('')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Video Metadata~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('~~~~~ Title       : %s' %(title))
print('~~~~~ Creator     : %s' %(creator))
print('~~~~~ Upload date : %s' %(upload_date))
print('~~~~~ Uploader    : %s' %(uploader))
print('~~~~~ Views       : %d' %(view_count))
print('~~~~~ Likes       : %d' %(like_count))
print('~~~~~ Dislikes    : %d' %(dislike_count))
print('~~~~~ ID          : %s' %(youtube_id))
print('~~~~~ Duration    : %s' %(duration))
print('~~~~~ Av. Rating  : %s' %(average_rating))
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Video Metadata~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~ Title       : Agüita e' Coco - Totoy El Frio x Beéle  [ Oficial Video ] 💦🥥
~~~~~ Creator     : Tygger Music, Beele
~~~~~ Upload date : 20191018
~~~~~ Uploader    : Totoy El Frio
~~~~~ Views       : 1299931
~~~~~ Likes       : 22747
~~~~~ Dislikes    : 515
~~~~~ ID          : XVNQSdAWVGo
~~~~~ Duration    : 207
~~~~~ Av. Rating  : 4.9114437
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


-------------

## 3. Frame/Clip decomposition

### 3.1 Clip decomposition

In [9]:
def get_video_length(filename):
    output = subprocess.check_output(("ffprobe", "-v", "error", "-show_entries", "format=duration", "-of",
                                      "default=noprint_wrappers=1:nokey=1", filename)).strip()
    video_length = int(float(output))
    print("Video length in seconds: " + str(video_length))

    return video_length


def ceildiv(a, b):
    return int(math.ceil(a / float(b)))


def split_by_seconds(filename, split_length, vcodec="copy", acodec="copy", extra="", video_length=None, **kwargs):
    if split_length and split_length <= 0:
        print("Split length can't be 0")
        raise SystemExit

    if not video_length:
        video_length = get_video_length(filename)
    split_count = ceildiv(video_length, split_length)
    if split_count == 1:
        print("Video length is less then the target split length.")
        raise SystemExit

    split_cmd = ["ffmpeg", "-i", filename, "-vcodec", vcodec, "-acodec", acodec] + shlex.split(extra)
    try:
        filebase = ".".join(filename.split(".")[:-1])
        fileext = filename.split(".")[-1]
    except IndexError as e:
        raise IndexError("No . in filename. Error: " + str(e))
    for n in range(0, split_count):
        split_args = []
        if n == 0:
            split_start = 0
        else:
            split_start = split_length * n

        split_args += ["-ss", str(split_start), "-t", str(split_length),
                       filebase + "-" + str(n + 1) + "-of-" + \
                       str(split_count) + "." + fileext]
        print("Decomposing clip: " + " ".join(split_cmd + split_args))
        subprocess.check_output(split_cmd + split_args)

In [10]:
split_by_seconds('XVNQSdAWVGo.mp4', 15)

Video length in seconds: 206
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 0 -t 15 XVNQSdAWVGo-1-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 15 -t 15 XVNQSdAWVGo-2-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 30 -t 15 XVNQSdAWVGo-3-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 45 -t 15 XVNQSdAWVGo-4-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 60 -t 15 XVNQSdAWVGo-5-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 75 -t 15 XVNQSdAWVGo-6-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 90 -t 15 XVNQSdAWVGo-7-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 105 -t 15 XVNQSdAWVGo-8-of-14.mp4
Decomposing clip: ffmpeg -i XVNQSdAWVGo.mp4 -vcodec copy -acodec copy -ss 120 -t 15 XVNQSdAWVGo-9-of-14.mp4
Decompo

### 3.2 Frame decomposition

In [1]:
def duration(video):
    command = 'ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1'
    return float(subprocess.check_output(command.split(' ') + [video]).strip())

In [6]:
video = "XVNQSdAWVGo.mp4"
seconds = 1
outformat = "jpg"
size = "1920x1200"
ffmpeg_args = []

ffmpeg_args += ['-s', size]

output_dir = os.path.join('frames')
os.makedirs(output_dir, exist_ok=True)

args = [i for i in ffmpeg_args]

basename = os.path.basename(video)
output = os.path.join(output_dir, basename)
os.makedirs(output, exist_ok=True)
output = os.path.join(output, f'frame%04d.{outformat}')

s = seconds
args += ['-ss', '3', '-r', f'1/{s}']

subprocess.run(['ffmpeg', '-i', f'{video}'] + args + [output])

CompletedProcess(args=['ffmpeg', '-i', 'XVNQSdAWVGo.mp4', '-s', '1920x1200', '-ss', '3', '-r', '1/1', 'frames/XVNQSdAWVGo.mp4/frame%04d.jpg'], returncode=0)

-----

## 4. NudeNet

In [15]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


frames = glob.glob("frames/XVNQSdAWVGo.mp4/*.jpg")
detector = NudeDetector('models/detector_model')
output_dir = os.path.join('detected')
os.makedirs(output_dir, exist_ok=True)
    

for frame in frames:
    img_path = frame
    result = detector.detect(img_path, min_prob=0.48)
    cv_image = cv2.imread(img_path)
    print("")
    print("")
    print("--------------------------------------------")
    for item in result:
        left = item['box'][0]
        top = item['box'][1]

        right = item['box'][2]
        bottom = item['box'][3]

        label = item['label']
        print(">>>>>>> I have detected: "+label)

        cv2.rectangle(cv_image, (left,top), (right,bottom), (0,0,255), 2)
        cv2.putText(cv_image, item['label'], (left,top), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    print("")
    print("")
    
    cv2.imwrite(output_dir+uuid.uuid1()+'.jpg',cv_image)

OSError: Unable to open file (file signature not found)